### T2 Linear programming with PuLP

PuLP is an LP modeler solving linear programming problems. You need to install PuLP before using. If you are using Anaconda/Miniconda, open the Conda Terminal and install PuLP:

    (base) C:\Users\YourName>conda activate env_tutorial
    (env_tutorial) C:\Users\YourName>conda install -c conda-forge pulp

In [1]:
import numpy as np
import pandas as pd
from pulp import *

To solve for an LP problem, the general steps are:

1. Create the model
2. Initiate decision variables
3. Add constraints
4. Add objective function
5. Specify solver and solve the model
6. Check the result

#### 1. Create the model.

In [2]:
model = LpProblem(name="myproblem", sense=LpMinimize)

#### 2. Initialize the decision variables.

In [3]:
H1 = LpVariable(name="H1", cat = "Integer", lowBound=0)
H2 = LpVariable(name="H2", cat = "Integer", lowBound=0)
H3 = LpVariable(name="H3", cat = "Integer", lowBound=0)

F1 = LpVariable(name="F1", cat = "Integer", lowBound=0)
F2 = LpVariable(name="F2", cat = "Integer", lowBound=0)
F3 = LpVariable(name="F3", cat = "Integer", lowBound=0)

W1 = LpVariable(name="W1", cat = "Integer", lowBound=0)
W2 = LpVariable(name="W2", cat = "Integer", lowBound=0)
W3 = LpVariable(name="W3", cat = "Integer", lowBound=0)

I1 = LpVariable(name="I1", lowBound=0)
I2 = LpVariable(name="I2", lowBound=0)
I3 = LpVariable(name="I3", lowBound=0)

B1 = LpVariable(name="B1", lowBound=0)
B2 = LpVariable(name="B2", lowBound=0)
B3 = LpVariable(name="B3", lowBound=0)

P1 = LpVariable(name="P1", lowBound=0)
P2 = LpVariable(name="P2", lowBound=0)
P3 = LpVariable(name="P3", lowBound=0)

In [4]:
# Input the demand data and workday data.
D1 = 500
D2 = 320
D3 = 400

T1 = 20
T2 = 16
T3 = 23

#### 3. Add the constraints to the model.

In [5]:
# Workforce balance constraint.
model += (W1 == 1200 + H1 - F1, "wf-const-1")
model += (W2 == W1 + H2 - F2, "wf-const-2")
model += (W3 == W2 + H3 - F3, "wf-const-3")

# Production/Capacity balance constraint.
model += (P1 == W1*T1*0.02, "prod-const-1")
model += (P2 == W2*T2*0.02, "prod-const-2")
model += (P3 == W3*T3*0.02, "prod-const-3")

# Inventory balance constraint.
model += (I1 - B1 == 0 - 0 + P1 - D1, "inv-const-1")
model += (I2 - B2 == I1 - B1 + P2 - D2, "inv-const-2")
model += (I3 - B3 == I2 - B2 + P3 - D3, "inv-const-3")

# End-of-period constraint.
model += (I3 - B3 >= 0, "inv-const-end")

#### 5.Add the objective function to the model.

In [6]:
# Input cost data.
cH = 1000
cF = 2000
cI = 500
cB = 2000
w = 100

# Define objective function.
obj_func = cH*(H1+H2+H3) + cF*(F1+F2+F3) + cI*(I1+I2+I3) + cB*(B1+B2+B3) + w*(T1*W1+T2*W2+T3*W3)
model += obj_func

In [7]:
# Check the model.
model

myproblem:
MINIMIZE
2000*B1 + 2000*B2 + 2000*B3 + 2000*F1 + 2000*F2 + 2000*F3 + 1000*H1 + 1000*H2 + 1000*H3 + 500*I1 + 500*I2 + 500*I3 + 2000*W1 + 1600*W2 + 2300*W3 + 0
SUBJECT TO
wf_const_1: F1 - H1 + W1 = 1200

wf_const_2: F2 - H2 - W1 + W2 = 0

wf_const_3: F3 - H3 - W2 + W3 = 0

prod_const_1: P1 - 0.4 W1 = 0

prod_const_2: P2 - 0.32 W2 = 0

prod_const_3: P3 - 0.46 W3 = 0

inv_const_1: - B1 + I1 - P1 = -500

inv_const_2: B1 - B2 - I1 + I2 - P2 = -320

inv_const_3: B2 - B3 - I2 + I3 - P3 = -400

inv_const_end: - B3 + I3 >= 0

VARIABLES
B1 Continuous
B2 Continuous
B3 Continuous
0 <= F1 Integer
0 <= F2 Integer
0 <= F3 Integer
0 <= H1 Integer
0 <= H2 Integer
0 <= H3 Integer
I1 Continuous
I2 Continuous
I3 Continuous
P1 Continuous
P2 Continuous
P3 Continuous
0 <= W1 Integer
0 <= W2 Integer
0 <= W3 Integer

#### 5. Solve the model.

In [8]:
# Specify the solver to be GLPK (optional)
path_to = r'C:\Users\June\miniconda3\envs\env_tutorial\Library\bin\glpsol.exe'
solver = GLPK_CMD(path=path_to)

# Solve the problem
status = model.solve(solver)

# status meaning: {0: 'Not Solved', 1: 'Optimal', -1: 'Infeasible', -2: 'Unbounded', -3: 'Undefined'}
status

1

#### 6. Check results.

In [9]:
print(f"Status: {model.status}, {LpStatus[model.status]}")
print(f"Minimum cost: {model.objective.value()}")
for var in model.variables():
     print(f"{var.name}: {var.value()}")

Status: 1, Optimal
Minimum cost: 6715850.0
B1: 20.0
B2: 36.32
B3: 0.0
F1: 0
F2: 251
F3: 0
H1: 0
H2: 0
H3: 0
I1: 0.0
I2: 0.0
I3: 0.22
P1: 480.0
P2: 303.68
P3: 436.54
W1: 1200
W2: 949
W3: 949
